## Imports

In [1]:
import pathlib

import pandas as pd
import plotly.express as px

## Environment variables

In [2]:
pd.options.plotting.backend = "plotly"

## Constants definition

In [3]:
WEATHER_PATH: pathlib.Path = pathlib.Path("../data/predict-energy-behavior-of-prosumers/historical_weather.csv")

## Data load

- **origin_date** - The date when the day-ahead prices became available.
- **forecast_date** - Represents the start of the 1-hour period when the price is valid
- **euros_per_mwh** - The price of electricity on the day ahead markets in euros per megawatt hour.
- **data_block_id** - All rows sharing the same `data_block_id` will be available at the same forecast time. This is a function of what information is available when forecasts are actually made, at 11 AM each morning. For example, if the forecast weather `data_block_id` for predictins made on October 31st is 100 then the historic weather `data_block_id` for October 31st will be 101 as the historic weather data is only actually available the next day.

In [11]:
weather: pd.DataFrame = pd.read_csv(WEATHER_PATH)
weather.head(10)

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id
0,2021-09-01 00:00:00,14.2,11.6,0.0,0.0,1015.9,31,31,0,11,7.083333,8,0.0,0.0,0.0,57.6,21.7,1.0
1,2021-09-01 00:00:00,13.9,11.5,0.0,0.0,1010.7,33,37,0,0,5.111111,359,0.0,0.0,0.0,57.6,22.2,1.0
2,2021-09-01 00:00:00,14.0,12.5,0.0,0.0,1015.0,31,34,0,0,6.333333,355,0.0,0.0,0.0,57.6,22.7,1.0
3,2021-09-01 00:00:00,14.6,11.5,0.0,0.0,1017.3,0,0,0,0,8.083333,297,358.0,277.0,81.0,57.6,23.2,1.0
4,2021-09-01 00:00:00,15.7,12.9,0.0,0.0,1014.0,22,25,0,0,8.416667,5,0.0,0.0,0.0,57.6,23.7,1.0
5,2021-09-01 00:00:00,16.0,11.4,0.0,0.0,1016.7,15,12,7,0,6.388889,297,349.0,274.0,75.0,57.6,24.2,1.0
6,2021-09-01 00:00:00,12.2,10.5,0.0,0.0,1005.3,19,7,0,41,4.111111,359,0.0,0.0,0.0,57.6,24.7,1.0
7,2021-09-01 00:00:00,12.2,10.5,0.0,0.0,1006.1,28,8,0,68,4.111111,354,0.0,0.0,0.0,57.6,25.2,1.0
8,2021-09-01 00:00:00,12.4,10.6,0.0,0.0,1003.8,49,26,0,85,4.250000,352,0.0,0.0,0.0,57.6,25.7,1.0
9,2021-09-01 00:00:00,12.9,11.0,0.0,0.0,1003.7,81,56,3,96,4.250000,351,0.0,0.0,0.0,57.6,26.2,1.0


In [5]:
weather.columns

Index(['datetime', 'temperature', 'dewpoint', 'rain', 'snowfall',
       'surface_pressure', 'cloudcover_total', 'cloudcover_low',
       'cloudcover_mid', 'cloudcover_high', 'windspeed_10m',
       'winddirection_10m', 'shortwave_radiation', 'direct_solar_radiation',
       'diffuse_radiation', 'latitude', 'longitude', 'data_block_id'],
      dtype='object')

In [6]:
weather.dtypes

datetime                   object
temperature               float64
dewpoint                  float64
rain                      float64
snowfall                  float64
surface_pressure          float64
cloudcover_total            int64
cloudcover_low              int64
cloudcover_mid              int64
cloudcover_high             int64
windspeed_10m             float64
winddirection_10m           int64
shortwave_radiation       float64
direct_solar_radiation    float64
diffuse_radiation         float64
latitude                  float64
longitude                 float64
data_block_id             float64
dtype: object

## EDA

### Check database info

In [10]:
weather.info(verbose=True, memory_usage="deep", show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1710802 entries, 0 to 1710801
Data columns (total 18 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   datetime                1710802 non-null  object 
 1   temperature             1710802 non-null  float64
 2   dewpoint                1710802 non-null  float64
 3   rain                    1710802 non-null  float64
 4   snowfall                1710802 non-null  float64
 5   surface_pressure        1710802 non-null  float64
 6   cloudcover_total        1710802 non-null  int64  
 7   cloudcover_low          1710802 non-null  int64  
 8   cloudcover_mid          1710802 non-null  int64  
 9   cloudcover_high         1710802 non-null  int64  
 10  windspeed_10m           1710802 non-null  float64
 11  winddirection_10m       1710802 non-null  int64  
 12  shortwave_radiation     1710802 non-null  float64
 13  direct_solar_radiation  1710802 non-null  float64
 14  di

In [8]:
weather.describe()

,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id
count,1.710802e+06,1.710802e+06,1.710802e+06,1.710802e+06,1.710802e+06,1.710802e+06,1.710802e+06,1.710802e+06,1.710802e+06,1.710802e+06,1.710802e+06,1.710802e+06,1.710802e+06,1.710802e+06,1.710802e+06,1.710802e+06,1.710802e+06
mean,5.740968e+00,2.240312e+00,4.962012e-02,1.604896e-02,1.009282e+03,6.091270e+01,4.668593e+01,3.440698e+01,3.605141e+01,4.849871e+00,1.978694e+02,1.064905e+02,6.445292e+01,4.203759e+01,5.865000e+01,2.495000e+01,3.192708e+02
std,8.025647e+00,7.224357e+00,2.079113e-01,7.462936e-02,1.308891e+01,3.776905e+01,4.074760e+01,3.832769e+01,4.135852e+01,2.475450e+00,8.993798e+01,1.799449e+02,1.334100e+02,6.195225e+01,6.873871e-01,2.015564e+00,1.837298e+02
min,-2.370000e+01,-2.590000e+01,0.000000e+00,0.000000e+00,9.429000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.760000e+01,2.170000e+01,1.000000e+00
25%,0.000000e+00,-2.600000e+00,0.000000e+00,0.000000e+00,1.001500e+03,2.500000e+01,3.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,1.390000e+02,0.000000e+00,0.000000e+00,0.000000e+00,5.790000e+01,2.320000e+01,1.600000e+02
50%,5.100000e+00,1.700000e+00,0.000000e+00,0.000000e+00,1.010400e+03,7.200000e+01,3.900000e+01,1.600000e+01,1.000000e+01,4.500000e+00,2.080000e+02,1.000000e+00,0.000000e+00,1.000000e+00,5.850000e+01,2.470000e+01,3.190000e+02
75%,1.120000e+01,7.200000e+00,0.000000e+00,0.000000e+00,1.018000e+03,1.000000e+02,9.400000e+01,7.200000e+01,8.500000e+01,6.277778e+00,2.630000e+02,1.400000e+02,4.700000e+01,7.400000e+01,5.910000e+01,2.670000e+01,4.780000e+02
max,3.260000e+01,2.380000e+01,1.680000e+01,2.660000e+00,1.049300e+03,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,2.175000e+01,3.600000e+02,8.490000e+02,7.540000e+02,3.880000e+02,5.970000e+01,2.820000e+01,6.370000e+02


## Conclusion

1. molto semplice, bisogna solo convertire le colonne